# Customer.IO Data Pipelines API - Objects and Relationships

## Purpose

This notebook demonstrates comprehensive object management and relationship tracking with Customer.IO's Data Pipelines API.
It covers groups (companies/organizations), object relationships, hierarchies, and advanced relationship patterns with proper validation and error handling.

## Prerequisites

- Complete setup from `00_setup_and_configuration.ipynb`
- Complete authentication setup from `01_authentication_and_utilities.ipynb`
- Customer.IO API key configured in Databricks secrets
- Sample data available in Delta tables

## Key Concepts

- **Groups/Objects**: Organizations, companies, accounts, or any business entity
- **Relationships**: Connections between users and groups (membership, ownership, etc.)
- **Hierarchies**: Parent-child relationships between objects
- **Attributes**: Custom properties for groups and relationships
- **Type Safety**: Validated object creation and management
- **Batch Operations**: Efficient bulk object and relationship management

## Object Types Covered

1. **Companies/Organizations**: Business entities with users
2. **Teams/Departments**: Sub-units within organizations
3. **Projects/Workspaces**: Collaborative spaces
4. **Custom Objects**: Business-specific entities
5. **Relationship Types**: Member, Admin, Owner, Custom roles

## Setup and Imports

In [ ]:
# Standard library imports
import sys
import os
from datetime import datetime, timezone, timedelta
from typing import Dict, List, Optional, Any, Union, Set
import json
import uuid
from enum import Enum
from dataclasses import dataclass

print("SUCCESS: Standard libraries imported")

In [ ]:
# Add utils directory to Python path
sys.path.append('/Workspace/Repos/customer_io_notebooks/utils')
print("SUCCESS: Utils directory added to Python path")

In [ ]:
# Import Customer.IO API utilities
from utils.api_client import CustomerIOClient
from utils.validators import (
    GroupRequest,
    validate_request_size,
    create_context
)

print("SUCCESS: Customer.IO API utilities imported")

In [ ]:
# Import transformation utilities
from utils.transformers import (
    CustomerTransformer,
    EventTransformer,
    BatchTransformer,
    ContextTransformer
)

print("SUCCESS: Transformation utilities imported")

In [ ]:
# Import error handling utilities
from utils.error_handlers import (
    CustomerIOError,
    RateLimitError,
    ValidationError,
    NetworkError,
    retry_on_error,
    ErrorContext
)

print("SUCCESS: Error handling utilities imported")

In [ ]:
# Import Databricks and Spark utilities
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from delta.tables import DeltaTable

print("SUCCESS: Databricks and Spark utilities imported")

In [ ]:
# Import validation and logging
import structlog
from pydantic import ValidationError as PydanticValidationError, BaseModel, Field, validator

# Import GroupManager for advanced object and relationship handling
from utils.group_manager import (
    GroupManager, 
    GroupTraits, 
    UserGroupRelationship, 
    ObjectHierarchy,
    ObjectType, 
    RelationshipRole, 
    RelationshipStatus,
    PermissionSet
)

# Initialize logger
logger = structlog.get_logger("objects_relationships")

print("SUCCESS: Validation, logging, and GroupManager imported")

## Configuration and Client Setup

In [ ]:
# Load configuration from setup notebook (secure approach)
try:
    CUSTOMERIO_REGION = dbutils.widgets.get("customerio_region") or "us"
    DATABASE_NAME = dbutils.widgets.get("database_name") or "customerio_demo"
    CATALOG_NAME = dbutils.widgets.get("catalog_name") or "main"
    ENVIRONMENT = dbutils.widgets.get("environment") or "test"
    
    print(f"Configuration loaded from setup notebook:")
    print(f"  Region: {CUSTOMERIO_REGION}")
    print(f"  Database: {CATALOG_NAME}.{DATABASE_NAME}")
    print(f"  Environment: {ENVIRONMENT}")
    
except Exception as e:
    print(f"WARNING: Could not load configuration from setup notebook: {str(e)}")
    print("INFO: Using fallback configuration")
    CUSTOMERIO_REGION = "us"
    DATABASE_NAME = "customerio_demo"
    CATALOG_NAME = "main"
    ENVIRONMENT = "test"

In [ ]:
# Get Customer.IO API key from secure storage
CUSTOMERIO_API_KEY = dbutils.secrets.get("customerio", "api_key")
print("SUCCESS: Customer.IO API key retrieved from secure storage")

In [ ]:
# Configure Spark to use the specified database
spark.sql(f"USE {CATALOG_NAME}.{DATABASE_NAME}")
print("SUCCESS: Database configured")

In [ ]:
# Initialize the Customer.IO client
try:
    client = CustomerIOClient(
        api_key=CUSTOMERIO_API_KEY,
        region=CUSTOMERIO_REGION,
        timeout=30,
        max_retries=3,
        retry_backoff_factor=2.0,
        enable_logging=True,
        spark_session=spark
    )
    print("SUCCESS: Customer.IO client initialized for object management")
    
except Exception as e:
    print(f"ERROR: Failed to initialize Customer.IO client: {str(e)}")
    raise

In [ ]:
# Initialize the GroupManager with the Customer.IO client
group_manager = GroupManager(client)
print("SUCCESS: GroupManager initialized for object and relationship management")

## Test-Driven Development: Object and Relationship Validation

In [ ]:
# Test basic group validation using GroupManager models

def test_basic_group_validation():
    """Test that basic groups have required fields and pass validation."""
    
    valid_group = {
        "userId": "user_123",
        "groupId": "company_456",
        "traits": {
            "name": "Acme Corporation",
            "industry": "Technology",
            "size": "100-500"
        },
        "timestamp": datetime.now(timezone.utc)
    }
    
    try:
        group_request = GroupRequest(**valid_group)
        assert group_request.userId == "user_123"
        assert group_request.groupId == "company_456"
        print("SUCCESS: Basic group validation test passed")
        return True
    except Exception as e:
        print(f"ERROR: Basic group validation test failed: {str(e)}")
        return False

# Run the test
test_basic_group_validation()

In [ ]:
# Test relationship validation using GroupManager models

def test_relationship_validation():
    """Test that relationship data structures are properly validated using GroupManager."""
    
    try:
        relationship = UserGroupRelationship(
            user_id="user_123",
            group_id="company_456",
            role=RelationshipRole.ADMIN,
            status=RelationshipStatus.ACTIVE,
            permissions=["users.manage", "billing.view"]
        )
        
        assert relationship.user_id == "user_123"
        assert relationship.group_id == "company_456"
        assert relationship.role == "admin"
        assert relationship.status == "active"
        print("SUCCESS: Relationship validation test passed")
        return True
    except Exception as e:
        print(f"ERROR: Relationship validation test failed: {str(e)}")
        return False

# Run the test
test_relationship_validation()

In [ ]:
# Test object types and enumerations from GroupManager
print(\"Available object types:\")\nfor obj_type in ObjectType:\n    print(f\"  - {obj_type.value}\")\n\nprint(\"\\nAvailable relationship roles:\")\nfor role in RelationshipRole:\n    print(f\"  - {role.value}\")\n\nprint(\"\\nAvailable relationship statuses:\")\nfor status in RelationshipStatus:\n    print(f\"  - {status.value}\")\n\nprint(\"\\nSUCCESS: Object types and enumerations are available from GroupManager\")"

In [ ]:
# Test type-safe group traits from GroupManager\nsample_company_traits = GroupTraits(\n    name=\"TechCorp Solutions\",\n    type=ObjectType.COMPANY,\n    industry=\"Software\",\n    size=\"50-100\",\n    website=\"techcorp.com\",\n    plan=\"enterprise\",\n    monthly_spend=4999.99,\n    employee_count=75\n)\n\nprint(\"GroupTraits model test:\")\nprint(f\"  Name: {sample_company_traits.name}\")\nprint(f\"  Type: {sample_company_traits.type}\")\nprint(f\"  Website: {sample_company_traits.website}\")  # Should auto-add https://\nprint(f\"  Employee Count: {sample_company_traits.employee_count}\")\nprint(\"SUCCESS: GroupTraits model working with validation\")"

In [ ]:
# Test relationship model from GroupManager\nsample_relationship = UserGroupRelationship(\n    user_id=\"user_admin_001\",\n    group_id=\"company_techcorp_001\",\n    role=RelationshipRole.ADMIN,\n    status=RelationshipStatus.ACTIVE,\n    permissions=[\"users.manage\", \"billing.view\", \"settings.edit\"],\n    metadata={\n        \"department\": \"Engineering\",\n        \"title\": \"VP of Engineering\"\n    }\n)\n\nprint(\"UserGroupRelationship model test:\")\nprint(f\"  User ID: {sample_relationship.user_id}\")\nprint(f\"  Group ID: {sample_relationship.group_id}\")\nprint(f\"  Role: {sample_relationship.role}\")\nprint(f\"  Status: {sample_relationship.status}\")\nprint(f\"  Permissions: {sample_relationship.permissions}\")\nprint(f\"  Joined at: {sample_relationship.joined_at}\")\nprint(\"SUCCESS: UserGroupRelationship model working with validation\")"

In [ ]:
# Test hierarchy model from GroupManager\nsample_hierarchy = ObjectHierarchy(\n    parent_id=\"company_techcorp_001\",\n    child_id=\"team_engineering_001\",\n    relationship_type=\"contains\",\n    level=1,\n    path=\"/company_techcorp_001/team_engineering_001\"\n)\n\nprint(\"ObjectHierarchy model test:\")\nprint(f\"  Parent ID: {sample_hierarchy.parent_id}\")\nprint(f\"  Child ID: {sample_hierarchy.child_id}\")\nprint(f\"  Relationship Type: {sample_hierarchy.relationship_type}\")\nprint(f\"  Level: {sample_hierarchy.level}\")\nprint(f\"  Path: {sample_hierarchy.path}\")\nprint(f\"  Created at: {sample_hierarchy.created_at}\")\nprint(\"SUCCESS: ObjectHierarchy model working with validation\")"

In [ ]:
# Test group creation using GroupManager

if ENVIRONMENT == "test":
    print("INFO: Running in test mode - group not actually sent")
    result = {"status": "test_success", "message": "Group validated successfully"}
else:
    result = group_manager.create_group(
        user_id="user_founder_001",
        group_id="company_techcorp_001",
        group_traits=sample_company_traits
    )

print("Group created using GroupManager:")
print(json.dumps({"traits": sample_company_traits.dict()}, indent=2, default=str))
print(f"Result: {result}")

In [ ]:
# Test permission system from GroupManager

print("Available default permissions by role:")
for role in RelationshipRole:
    permissions = PermissionSet.get_permissions(role)
    print(f"  {role.value}: {permissions}")

print("\nTesting permission checking:")
admin_perms = PermissionSet.get_permissions(RelationshipRole.ADMIN)
print(f"Admin has 'users.manage': {PermissionSet.has_permission(admin_perms, 'users.manage')}")
print(f"Admin has 'users.delete': {PermissionSet.has_permission(admin_perms, 'users.delete')}")

print("SUCCESS: Permission system from GroupManager working correctly")

In [ ]:
# Test hierarchy validation using GroupManager models

def test_hierarchy_validation():
    """Test that hierarchy models are properly validated."""
    
    try:
        hierarchy = ObjectHierarchy(
            parent_id="company_techcorp_001",
            child_id="team_engineering_001",
            relationship_type="contains",
            level=1,
            path="/company_techcorp_001/team_engineering_001"
        )
        
        assert hierarchy.parent_id == "company_techcorp_001"
        assert hierarchy.child_id == "team_engineering_001"
        assert hierarchy.level == 1
        print("SUCCESS: Hierarchy validation test passed")
        return True
    except Exception as e:
        print(f"ERROR: Hierarchy validation test failed: {str(e)}")
        return False

# Run the test
test_hierarchy_validation()

## Group/Object Management Implementation

In [ ]:
# Test group creation with GroupTraits

sample_company = GroupTraits(
    name="TechCorp Solutions",
    type=ObjectType.COMPANY,
    industry="Software",
    size="50-100",
    website="techcorp.com",
    plan="enterprise",
    monthly_spend=4999.99,
    employee_count=75
)

print("Created GroupTraits for TechCorp Solutions:")
print(f"  Name: {sample_company.name}")
print(f"  Type: {sample_company.type}")
print(f"  Website: {sample_company.website}")  # Should auto-add https://
print(f"  Employee Count: {sample_company.employee_count}")
print("SUCCESS: GroupTraits creation and validation working")

In [ ]:
# Send group using GroupManager

if ENVIRONMENT == "test":
    print("INFO: Running in test mode - group not actually sent")
    result = {"status": "test_success", "message": "Group validated successfully"}
else:
    result = group_manager.create_group(
        user_id="user_founder_001",
        group_id="company_techcorp_001",
        group_traits=sample_company
    )

print(f"Group creation result: {result}")
print("SUCCESS: Group created using GroupManager")

## Relationship Management

In [ ]:
# Create user-group relationship using GroupManager

admin_relationship = UserGroupRelationship(
    user_id="user_admin_001",
    group_id="company_techcorp_001",
    role=RelationshipRole.ADMIN,
    status=RelationshipStatus.ACTIVE,
    permissions=["users.manage", "billing.view", "settings.edit"],
    metadata={
        "department": "Engineering",
        "title": "VP of Engineering"
    }
)

if ENVIRONMENT == "test":
    print("INFO: Running in test mode - relationship not actually sent")
    result = {"status": "test_success", "message": "Relationship validated successfully"}
else:
    result = group_manager.create_relationship(admin_relationship)

print("Admin relationship created:")
print(f"  User: {admin_relationship.user_id}")
print(f"  Group: {admin_relationship.group_id}")
print(f"  Role: {admin_relationship.role}")
print(f"  Permissions: {admin_relationship.permissions}")
print(f"Result: {result}")

In [ ]:
# Update relationship status using GroupManager

if ENVIRONMENT == "test":
    print("INFO: Running in test mode - status update not actually sent")
    result = {"status": "test_success", "message": "Status update validated successfully"}
else:
    result = group_manager.update_relationship_status(
        user_id="user_member_002",
        group_id="company_techcorp_001",
        new_status=RelationshipStatus.SUSPENDED,
        previous_status=RelationshipStatus.ACTIVE,
        reason="Payment failure"
    )

print("Relationship status update:")
print(f"  User: user_member_002")
print(f"  Group: company_techcorp_001")
print(f"  New Status: suspended")
print(f"  Reason: Payment failure")
print(f"Result: {result}")

## Hierarchical Relationships

In [ ]:
# Create object hierarchy using GroupManager

engineering_team = GroupTraits(
    name="Engineering Team",
    type=ObjectType.TEAM,
    size="10-20",
    created_at=datetime.now(timezone.utc)
)

# Create the team group first
if ENVIRONMENT == "test":
    print("INFO: Running in test mode - team group not actually sent")
    team_result = {"status": "test_success"}
else:
    team_result = group_manager.create_group(
        user_id="user_team_lead_001",
        group_id="team_engineering_001",
        group_traits=engineering_team
    )

print(f"Engineering team created: {team_result}")

# Create hierarchy relationship
team_hierarchy = ObjectHierarchy(
    parent_id="company_techcorp_001",
    child_id="team_engineering_001",
    relationship_type="contains",
    level=1,
    path="/company_techcorp_001/team_engineering_001"
)

# Create hierarchy events using GroupManager
hierarchy_events = group_manager.create_hierarchy(
    hierarchy=team_hierarchy,
    parent_type=ObjectType.COMPANY,
    child_type=ObjectType.TEAM
)

print(f"\nCreated {len(hierarchy_events)} hierarchy events:")
for event in hierarchy_events:
    print(f"  - {event['event']} for {event['userId']}")

## Batch Operations for Groups and Relationships

In [ ]:
# Create sample departments using GroupTraits

departments = []

# Sales Department
sales_dept = GroupTraits(
    name="Sales Department",
    type=ObjectType.DEPARTMENT,
    size="20-30",
    employee_count=25
)
departments.append({
    "userId": "user_dept_head_001",
    "groupId": "dept_sales_001",
    "traits": sales_dept.dict(exclude_none=True),
    "timestamp": datetime.now(timezone.utc)
})

# Marketing Department  
marketing_dept = GroupTraits(
    name="Marketing Department", 
    type=ObjectType.DEPARTMENT,
    size="10-20",
    employee_count=15
)
departments.append({
    "userId": "user_dept_head_002",
    "groupId": "dept_marketing_001", 
    "traits": marketing_dept.dict(exclude_none=True),
    "timestamp": datetime.now(timezone.utc)
})

# Product Department
product_dept = GroupTraits(
    name="Product Department",
    type=ObjectType.DEPARTMENT, 
    size="5-10",
    employee_count=8
)
departments.append({
    "userId": "user_dept_head_003",
    "groupId": "dept_product_001",
    "traits": product_dept.dict(exclude_none=True), 
    "timestamp": datetime.now(timezone.utc)
})

print(f"Created {len(departments)} department definitions:")
for dept in departments:
    print(f"  - {dept['traits']['name']} ({dept['groupId']})")

In [ ]:
# Create sample team relationships using GroupManager models

team_relationships = [
    UserGroupRelationship(
        user_id="user_eng_001",
        group_id="team_engineering_001",
        role=RelationshipRole.MEMBER,
        permissions=["code.write", "pr.create"],
        metadata={"seniority": "senior", "specialization": "backend"}
    ),
    UserGroupRelationship(
        user_id="user_eng_002",
        group_id="team_engineering_001",
        role=RelationshipRole.MEMBER,
        permissions=["code.write", "pr.create"],
        metadata={"seniority": "mid", "specialization": "frontend"}
    ),
    UserGroupRelationship(
        user_id="user_eng_003",
        group_id="team_engineering_001",
        role=RelationshipRole.MEMBER,
        permissions=["code.read"],
        metadata={"seniority": "junior", "specialization": "fullstack"}
    )
]

print(f"Created {len(team_relationships)} team relationships:")
for rel in team_relationships:
    print(f"  - {rel.user_id} -> {rel.group_id} as {rel.role} ({rel.metadata.get('seniority', 'unknown')})")

In [ ]:
# Send batch operations using GroupManager

if ENVIRONMENT == "test":
    print("INFO: Running in test mode - batch operations not actually sent")
    
    # Simulate batch results
    batch_groups_result = [
        {"batch_id": 0, "status": "test_success", "count": len(departments)}
    ]
    
    batch_relationships_result = [
        {"batch_id": 0, "status": "test_success", "count": len(team_relationships)}
    ]
    
else:
    # Send departments batch using GroupManager
    batch_groups_result = group_manager.batch_create_groups(departments)
    
    # Send relationships batch using GroupManager  
    batch_relationships_result = group_manager.batch_create_relationships(team_relationships)

print("Batch groups results:")
for result in batch_groups_result:
    print(f"  Batch {result['batch_id']}: {result['status']} ({result['count']} groups)")

print("\nBatch relationships results:")
for result in batch_relationships_result:
    print(f"  Batch {result['batch_id']}: {result['status']} ({result['count']} relationships)")

## Complex Relationship Queries

In [ ]:
# Find users by role using GroupManager

all_relationships = team_relationships + [admin_relationship]

# Use GroupManager method to get users by role
members = group_manager.get_users_by_role(
    group_id="team_engineering_001",
    role=RelationshipRole.MEMBER,
    relationships=all_relationships
)

admins = group_manager.get_users_by_role(
    group_id="team_engineering_001", 
    role=RelationshipRole.ADMIN,
    relationships=all_relationships
)

print(f"Found {len(members)} members in Engineering Team:")
for user_id in members:
    print(f"  - {user_id}")

print(f"\nFound {len(admins)} admins in Engineering Team:")
for user_id in admins:
    print(f"  - {user_id}")

print("SUCCESS: User queries using GroupManager working correctly")

In [ ]:
# Build hierarchy tree using GroupManager

# Create additional hierarchies for testing
additional_hierarchies = [
    team_hierarchy,
    ObjectHierarchy(
        parent_id="company_techcorp_001",
        child_id="dept_sales_001",
        relationship_type="contains",
        level=1,
        path="/company_techcorp_001/dept_sales_001"
    ),
    ObjectHierarchy(
        parent_id="dept_sales_001",
        child_id="team_sales_na_001",
        relationship_type="contains",
        level=2,
        path="/company_techcorp_001/dept_sales_001/team_sales_na_001"
    )
]

# Build the tree using GroupManager
org_tree = group_manager.build_hierarchy_tree(
    root_id="company_techcorp_001",
    hierarchies=additional_hierarchies
)

print("Organization hierarchy tree:")
print(json.dumps(org_tree, indent=2))
print("SUCCESS: Hierarchy tree built using GroupManager")

## Data-Driven Group Generation from Spark

In [ ]:
# Load sample organization data
print("=== Data-Driven Group Generation ===")

# Create sample organizations table if it doesn't exist
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {CATALOG_NAME}.{DATABASE_NAME}.organizations (
    org_id STRING,
    org_name STRING,
    industry STRING,
    employee_count INT,
    annual_revenue DOUBLE,
    created_at TIMESTAMP
) USING DELTA
""")

# Insert sample data
spark.sql(f"""
INSERT INTO {CATALOG_NAME}.{DATABASE_NAME}.organizations
SELECT * FROM VALUES
    ('org_001', 'Alpha Corp', 'Finance', 250, 5000000, current_timestamp()),
    ('org_002', 'Beta Industries', 'Manufacturing', 500, 10000000, current_timestamp()),
    ('org_003', 'Gamma Tech', 'Technology', 100, 2500000, current_timestamp())
WHERE NOT EXISTS (
    SELECT 1 FROM {CATALOG_NAME}.{DATABASE_NAME}.organizations WHERE org_id = 'org_001'
)
""")

# Load organizations
organizations_df = spark.table(f"{CATALOG_NAME}.{DATABASE_NAME}.organizations")
organizations_df.show()

In [ ]:
# Transform Spark data to Customer.IO groups using GroupTraits

def transform_orgs_to_groups(df):
    """Transform organization data to Customer.IO group format using GroupTraits."""
    
    # Collect data (in production, process in batches)
    orgs = df.collect()
    
    groups = []
    for org in orgs:
        # Determine organization size category
        emp_count = org['employee_count']
        if emp_count < 50:
            size = "1-50"
        elif emp_count < 200:
            size = "50-200"
        elif emp_count < 500:
            size = "200-500"
        else:
            size = "500+"
        
        # Create GroupTraits for validation
        group_traits = GroupTraits(
            name=org['org_name'],
            type=ObjectType.COMPANY,
            industry=org['industry'],
            size=size,
            employee_count=emp_count,
            created_at=org['created_at']
        )
        
        group_data = {
            "userId": f"admin_{org['org_id']}",  # Default admin user
            "groupId": org['org_id'],
            "traits": group_traits.dict(exclude_none=True),
            "timestamp": datetime.now(timezone.utc)
        }
        
        groups.append(group_data)
    
    return groups

# Transform organizations using GroupTraits
spark_groups = transform_orgs_to_groups(organizations_df)
print(f"Transformed {len(spark_groups)} organizations to groups using GroupTraits")

# Show sample
if spark_groups:
    print("\nSample transformed group:")
    print(json.dumps(spark_groups[0], indent=2, default=str))

In [ ]:
# Process Spark groups using GroupManager

if spark_groups:
    if ENVIRONMENT == "test":
        print("INFO: Running in test mode - Spark groups not actually sent")
        spark_batch_results = [
            {"batch_id": 0, "status": "test_success", "count": len(spark_groups)}
        ]
    else:
        # Send batch using GroupManager
        spark_batch_results = group_manager.batch_create_groups(spark_groups)
    
    print("\nSpark groups batch results:")
    for result in spark_batch_results:
        print(f"  Batch {result['batch_id']}: {result['status']} ({result['count']} groups)")
else:
    print("No organizations to process")

## Permission Management

In [ ]:
# Test permission system from GroupManager

print("Testing GroupManager permission system:")

print("\nDefault permissions by role:")
for role in RelationshipRole:
    permissions = PermissionSet.get_permissions(role)
    print(f"  {role.value}: {permissions}")

print("\nTesting permission checking:")
admin_perms = PermissionSet.get_permissions(RelationshipRole.ADMIN)
member_perms = PermissionSet.get_permissions(RelationshipRole.MEMBER)

print(f"Admin can manage users: {PermissionSet.has_permission(admin_perms, 'users.manage')}")
print(f"Admin can view billing: {PermissionSet.has_permission(admin_perms, 'billing.view')}")
print(f"Member can manage users: {PermissionSet.has_permission(member_perms, 'users.manage')}")
print(f"Member can view users: {PermissionSet.has_permission(member_perms, 'users.view')}")

print("SUCCESS: GroupManager permission system working correctly")

In [ ]:
# Update user permissions using GroupManager

if ENVIRONMENT == "test":
    print("INFO: Running in test mode - permission update not actually sent")
    result = {"status": "test_success", "message": "Permission update validated successfully"}
else:
    result = group_manager.update_user_permissions(
        user_id="user_eng_003",
        group_id="team_engineering_001",
        new_permissions=["users.view", "code.write", "pr.create"],
        previous_permissions=["users.view"],
        added=["code.write", "pr.create"],
        removed=[]
    )

print("Permission update using GroupManager:")
print(f"  User: user_eng_003")
print(f"  Group: team_engineering_001")
print(f"  Added permissions: ['code.write', 'pr.create']")
print(f"Result: {result}")
print("SUCCESS: Permission update using GroupManager")

## Performance Monitoring and Metrics

In [ ]:
# Calculate organizational metrics using GroupManager

# Collect all groups for metrics
all_groups = []

# Add main company
all_groups.append({
    "traits": sample_company.dict(exclude_none=True)
})

# Add engineering team  
all_groups.append({
    "traits": engineering_team.dict(exclude_none=True)
})

# Add departments
for dept in departments:
    all_groups.append({
        "traits": dept["traits"]
    })

# Add Spark groups
all_groups.extend([{"traits": group["traits"]} for group in spark_groups])

# Calculate metrics using GroupManager
metrics = group_manager.calculate_org_metrics(
    groups=all_groups,
    relationships=all_relationships,
    hierarchies=additional_hierarchies
)

print("=== Organization Metrics (using GroupManager) ===")
print(json.dumps(metrics, indent=2))
print("SUCCESS: Metrics calculated using GroupManager")

## Clean Up and Summary

In [ ]:
# Final summary
print("=== Objects and Relationships Summary ===")

print("\n=== Objects Created ===")
print("SUCCESS: Company organization (TechCorp Solutions)")
print("SUCCESS: Engineering team with hierarchy")
print("SUCCESS: Multiple departments (Sales, Marketing, Product)")
print("SUCCESS: Organizations from Spark data")

print("\n=== Relationships Established ===")
print("SUCCESS: User-group relationships with roles")
print("SUCCESS: Hierarchical parent-child relationships")
print("SUCCESS: Permission-based access control")
print("SUCCESS: Status management (active, suspended, etc.)")

print("\n=== Key Capabilities Demonstrated ===")
print("SUCCESS: Type-safe object creation with validation")
print("SUCCESS: Complex relationship management")
print("SUCCESS: Hierarchical organization structures")
print("SUCCESS: Permission-based access control")
print("SUCCESS: Batch operations with optimization")
print("SUCCESS: Data transformation from Spark")
print("SUCCESS: Organizational metrics and analytics")

In [ ]:
# Close the API client connection
client.close()
print("SUCCESS: API client connection closed")

print("\nCOMPLETED: Objects and relationships notebook finished successfully!")
print("Ready for device management operations in the next notebook.")